# **LangChain: Agents**

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [2]:
# ! pip install langchain langchain_groq langchain_community langchain_experimental

Next, we need to set the `Groq_API_KEY`:

In [3]:
import os

os.environ["GROQ_API_KEY"] = "YOUR-API-KEY"

In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model = "llama3-70b-8192",
    temperature = 0.0
)


We want our agent to use Wikipedia as a resource:

In [6]:
## ! pip install -U wikipedia

In [8]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType

## Initialize the agent tools
tools = load_tools(
    tool_names = ["llm-math", "wikipedia"],
    llm = llm
)

## Create the agent
agent = initialize_agent(
    tools,
    llm,
    ## AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION. This type indicates that the agent will use a "zero-shot" approach to react to descriptions,
    ## meaning it can handle tasks without needing extensive prior training on specific examples.
    agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors = True,
    verbose = True
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


### **Using LLM-Math**

In [9]:
agent("What is the 25% of 300, output just a single number?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Question: What is the 25% of 300, output just a single number?

Thought: I need to calculate 25% of 300.

Action:
```
{
  "action": "Calculator",
  "action_input": "0.25 * 300"
}
```

Observation: Answer: 75.0
Thought:I now know the final answer
Final Answer: 75

> Finished chain.


{'input': 'What is the 25% of 300, output just a single number?',
 'output': '75'}

### **Using Wikipedia**

In [10]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"

agent(question)



> Entering new AgentExecutor chain...
Question: What book did Tom M. Mitchell write?

Thought: I need to search for information about Tom M. Mitchell to find out what book he wrote.

Action:
```
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```


/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Tom Mitchell (Australian footballer)
Summary: Thomas Mitchell (born 31 May 1993) is a professional Australian rules footb

{'input': 'Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU)what book did he write?',
 'output': 'Machine Learning'}

## **Python Agent**
Our current agent with its own tools `llm-math` and `wikipedia`, is not able to run a python code.

In [13]:
customer_list = [["Harrison", "Chase"],
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"],
                 ["Geoff","Fusion"],
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

## This code results to an error:
# agent(f"""Sort these customers by \
# last name and then first name \
# and print the output: {customer_list}""")

So, we need to create a new agent with more advanced tools:

In [21]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool

agent = initialize_agent(
    [PythonREPLTool()],
    llm,
    agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors = True,
    verbose = True
)

customer_list = [["Harrison", "Chase"],
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"],
                 ["Geoff","Fusion"],
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

agent(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")



> Entering new AgentExecutor chain...
Thought: I need to sort the list of customers by last name and then first name.

Action:
```
{
  "action": "Python_REPL",
  "action_input": "customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]; customers.sort(key=lambda x: (x[1], x[0])); print(customers)"
}
```

Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

Thought:I need to print the sorted list.

Action:
```
{
  "action": "Python_REPL",
  "action_input": "print([['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']])"
}
```


Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

Thought:I already have the sorted l

{'input': "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
 'output': "[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"}

We can see more details that how informations is being proccessed by using the **debug** mode.

In [22]:
import langchain

langchain.debug = True

agent(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")

langchain.debug = False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatGroq] Entering LLM run with input:
{
  "prompts": [
    "System: Answer the following questions as best you can. You have access to the following tools:\n\nPython_REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. I

## **Define our own tool:**

So far, we have used the predefined tools. now, we will define our own tools.

In [24]:
# ! pip install DateTime

In [25]:
from langchain.agents import tool
from datetime import date

@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [26]:
agent = initialize_agent(
    tools + [time],
    llm,
    agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors = True,
    verbose = True)

In [27]:
agent("whats the date today?")



> Entering new AgentExecutor chain...
Question: whats the date today?

Thought: I need to get the current date.

Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2024-06-20
Thought:I now know the final answer
Final Answer: 2024-06-20

> Finished chain.


{'input': 'whats the date today?', 'output': '2024-06-20'}

### **Agents need corresponding tools !!!**
If we don't define the tools for our agents, they are not able to answer corresponding questions. note that they are not the same as llms.

In [29]:
tools = load_tools(["llm-math"], llm = llm)

agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors = True,
    verbose = True
)

agent("Do you know Isabela Merced?")



> Entering new AgentExecutor chain...
Question: Do you know Isabela Merced?

Thought: I'm not familiar with the name Isabela Merced, but I can try to find out who she is.

Action:
```
{
  "action": "None",
  "action_input": "Isabela Merced"
}
```

Observation: None is not a valid tool, try one of [Calculator].
Thought:I apologize for the mistake! Since I don't have any information about Isabela Merced, I'll try to find out who she is.

Thought: I'll try to find information about Isabela Merced online.

Action:
```
{
  "action": "None",
  "action_input": "Isabela Merced"
}
```


Observation: None is not a valid tool, try one of [Calculator].
Thought:I apologize for the mistake again! Since I don't have any information about Isabela Merced, I'll try to find out who she is.

Action:
```
{
  "action": "None",
  "action_input": "Isabela Merced"
}
```


Observation: None is not a valid tool, try one of [Calculator].
Thought:I apologize for the mistake again! Since I don't have any informat

ValueError: unknown format from LLM: I think there might be some mistake! Isabela Merced is an American actress and singer, not a math problem. Could you please provide a math problem or expression that I can help you with?

If you meant to ask a math question, please feel free to ask, and I'll do my best to assist you!

# **Why Agents?**

Let's delve into why we might define agents with specific tools and not just using the pre-trained LLM and the trade-offs involved.

## **Reasons for Using Agents with Specific Tools**

- ### **Specialization and Accuracy:**
   - **Expertise in Specific Domains:** Agents with specific tools can provide more accurate and reliable information in certain domains. For example, an agent using a tool designed for accessing Wikipedia can ensure it retrieves and presents the most relevant and up-to-date information from Wikipedia.

  - **Specialized Processing:** Some tasks require specialized processing that general LLMs might not handle well. For example, a math tool can accurately perform complex calculations that a general LLM might struggle with.

- ### **Resource Management:**
    - **Efficiency:** Using specialized tools can be more efficient than relying on a general LLM to handle everything. It reduces the load on the LLM and can lead to faster response times.
    - **Cost:** Specialized tools can be more cost-effective for certain tasks, especially if they are less resource-intensive than a general LLM.

- ### **Error Handling and Fallback Mechanisms:**
    - **Graceful Degradation:** If a tool-specific agent encounters a situation where it cannot find the required information, it can gracefully handle the error or escalate it to another agent or system designed to manage such cases.
    - **Fallbacks:** You can design a system where if one tool fails to provide the necessary information, another tool or agent is used as a fallback.


## **Why Not Just Use a Pre-Trained LLM?**

- ### **Generalization Limits:**
    - **Breadth vs. Depth:** While pre-trained LLMs are excellent at generalizing across a wide range of topics, they may lack depth in specific domains. Specialized tools can provide the necessary depth and precision.
- ### **Current Knowledge Limitations:**
    - **Static Knowledge:** A pre-trained LLM has a fixed knowledge base up to its training cutoff. It won't have access to the latest information unless explicitly updated. In contrast, an agent using a tool like Wikipedia can access up-to-date information dynamically.

- ### **Complex Tasks:**
    - **Complex Workflows:** Some tasks involve multiple steps or require integrating information from various sources. Agents can be designed to handle such workflows more effectively than a general LLM, which might not be able to chain tasks or integrate data from multiple tools seamlessly.